In [69]:
import pandas as pd
import numpy as np
from chefboost.training import Training
from sklearn import datasets
df = pd.read_csv("wine_dataset.csv")
df.rename(columns={"type":"Decision"},inplace=True)
config = {"algorithm":"ID3"}
gains = Training.findGains(df,config)
print(gains)
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]

{'gains': {'citric acid': 0.11644331159277607, 'residual sugar': 0.23005216511474283, 'pH': 0.0964537780661524, 'sulphates': 0.17530932492038187, 'alcohol': 0.00873819106050372}, 'entropy': 1.0}


In [90]:
y = pd.DataFrame({'type':[1,0,0,0,0,1,1,1,0,0]})
column = y.keys()[-1]
print(y[column].iloc[0])

1


In [71]:
X = pd.DataFrame({"data":[0,0,0,0],"type":["a","B","a","a"]})
y = pd.DataFrame({"data":[1,2,3,4]})
data_is_equal = True
for key in X.keys():
    if len(X[key].unique()) == 1:
        continue
    else:
        data_is_equal = False
print(data_is_equal)

False


In [72]:
#y = [0,0,0,0,0,0,0,0,0,0]
#y = [1,1,1,1,1,1,1,1,1,1]
y = [0,1,1,1,1,0]
if len(set(y)) == 1:
    print(y[0])

In [73]:
class Node():
        def __init__(self,dec_key,dec_val):
            self.dec_key = dec_key
            self.dec_val = dec_val
        def right(self,next):
            self.right_branch = next
        def left(self,next):
            self.left_branch = next
        def set_prev(self,prev):
             self.prev = prev
        pass
class Leaf():
    def __init__(self,val):
        self.val = val
        pass
    def set_prev(self,prev):
         self.prev = prev


In [74]:
def calculate_data_entropy(y):
    data_entropy = 0
    column = y.keys()[-1]
    for label in y[column].unique():
        label_prob = (y[column].value_counts()[label])/len(y)
        data_entropy -= (label_prob) * np.log2(label_prob)
    return data_entropy

In [75]:
def impurity(X,y,impurity_measure):
    impurity_list = []
    for key in X.keys():
        split = np.mean(X[key])
        X_split = X[key] < split
        impurity = 0
        for value in X_split.unique():
            prob_X = X_split.value_counts()[value]/len(X_split)
            if impurity_measure == 'gini':
                impurity += prob_X
            column = y.keys()[-1]
            for label in y[column].unique():
                try:
                    prob_y_cond = (((X_split == value)&(y[column] == label)).value_counts()[True]/len(X_split))/prob_X
                except:
                    impurity_list.append(0)
                    continue
                if impurity_measure == 'entropy':
                    impurity -= prob_X * prob_y_cond * np.log2(prob_y_cond)
                elif impurity_measure == 'gini':
                    impurity -= prob_X * (prob_y_cond**2)
        impurity_list.append(impurity)
    impurity_list = np.array(impurity_list)
    print(impurity_list)


In [ ]:
df = pd.read_csv("wine_dataset.csv")
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]

impurity(X,y,'gini')

In [76]:
def information_gain(X,y):
    #calculate entropy of X data set
    data_entropy = calculate_data_entropy(y)
    #calculate conditional entropy given a split data for each predictor
    cond_entropy_array = calculate_conditional_entropy(X,y)
    if cond_entropy_array.all() == 0:
        return Leaf(y[y.keys()[-1]].value_counts()[0])
    #difference in data set entropy and conditional entropies gives us information gain of each split
    information_gain = data_entropy - cond_entropy_array
    print(information_gain)
    return np.argmax(information_gain)


In [77]:
def learn(X,y,prev = None,impurity_measure = 'entropy'):
    print(X.info())
    print(y)
    #check if all labels are the same; return leaf of that value if they are
    column = y.keys()[-1]
    if len(y[column].unique()) == 1:
        branch = Leaf(y[column].iloc[0])
        branch.set_prev(prev)
        return branch
    #check if all data values are the same and return most common label if true
    data_is_equal = True
    for key in X.keys():
        if len(X[key].unique()) == 1:
            continue
        else:
            data_is_equal = False
    if data_is_equal:
        branch = Leaf(y[column].value_counts()[0])
        branch.set_prev(prev)
        return branch
    
    #find split with most information gain and split data to left and right branches
    else:
        split_index = information_gain(X,y)
        if isinstance(split_index,Leaf):
            split_index.set_prev(prev)
            return split_index
        split_key = X.keys()[split_index]
        split_value = np.mean(X[split_key])
        branch = Node(split_key,split_value)
        branch.set_prev(prev)
        branch.right = learn(X[X[split_key] < split_value],y[X[split_key] < split_value])
        branch.left = learn(X[X[split_key] >= split_value],y[X[split_key] >= split_value])
        return branch

In [78]:
df = pd.read_csv("wine_dataset.csv")
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]

tree = learn(X,y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3198 entries, 0 to 3197
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   citric acid     3198 non-null   float64
 1   residual sugar  3198 non-null   float64
 2   pH              3198 non-null   float64
 3   sulphates       3198 non-null   float64
 4   alcohol         3198 non-null   float64
dtypes: float64(5)
memory usage: 125.0 KB
None
      type
0        1
1        1
2        1
3        0
4        1
...    ...
3193     0
3194     0
3195     1
3196     0
3197     0

[3198 rows x 1 columns]
[0.00917571 0.23005217 0.09645378 0.17530932 0.00128067]


<class 'pandas.core.frame.DataFrame'>
Index: 2202 entries, 0 to 3195
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   citric acid     2202 non-null   float64
 1   residual sugar  2202 non-null   float64
 2   pH              2202 non-null   float64
 3   sulphates       2202 non-null   float64
 4   alcohol         2202 non-null   float64
dtypes: float64(5)
memory usage: 103.2 KB
None
      type
0        1
1        1
2        1
4        1
5        1
...    ...
3187     0
3189     1
3191     0
3193     0
3195     1

[2202 rows x 1 columns]
[0.03169458 0.03175836 0.05321002 0.09521774 0.03077982]
<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, 0 to 3193
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   citric acid     1260 non-null   float64
 1   residual sugar  1260 non-null   float64
 2   pH              1260 non-null   float